In [ ]:
# required modules
%matplotlib inline
matplotlib.style.use('default')
import pandas as pd
import matplotlib
import requests
from IPython.display import JSON

In [49]:
url = "https://api.foursquare.com/v3/places/search"
base = "46.205,6.143" #Geneva Center

headers = {
    "Accept": "application/json",
    "Authorization": "fsq3V3VJ3Eo+Pxwu5vUFPK9qfRQiPyRblc7ovL8NXNqZIvg="
}

par = {
    'll':base,
    'radius':100,
    'category':13065,
    'limit':50,
    'fields':"name,fsq_id,categories,price,rating,stats,location,geocodes"
}

response = requests.get(url, headers=headers, params=par).json()

In [53]:
response['results']

[{'fsq_id': '518635f0498e1c0b6eb9b1b6',
  'categories': [{'id': 11045,
    'name': 'Bank',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/financial_',
     'suffix': '.png'}}],
  'geocodes': {'main': {'latitude': 46.204841, 'longitude': 6.142857},
   'roof': {'latitude': 46.204841, 'longitude': 6.142857}},
  'location': {'country': 'CH',
   'cross_street': '',
   'formatted_address': '',
   'region': 'Genève'},
  'name': 'National Bank Of Abu Dhabi'},
 {'fsq_id': '494a9c86641b44b6006c1b86',
  'categories': [{'id': 17051,
    'name': 'Watch Store',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_',
     'suffix': '.png'}}],
  'geocodes': {'main': {'latitude': 46.2049, 'longitude': 6.142894},
   'roof': {'latitude': 46.2049, 'longitude': 6.142894}},
  'location': {'address': 'Rue des Moulins 1',
   'country': 'CH',
   'formatted_address': 'Rue des Moulins 1, 1204 Genève',
   'locality': 'Genève',
   'postcode': '1204',
   'region': 'Genè

In [58]:
tdf = pd.json_normalize(response['results'])[[\
                                       'fsq_id',
                                       'categories',
                                       'name',
                                       'price',
                                       'rating',
                                       'stats.total_ratings',
                                       'geocodes.main.latitude',
                                       'geocodes.main.longitude',
                                       'location.formatted_address',
                                       'location.locality',
                                       #'location.neighborhood',
                                       'location.postcode',
                                       'location.region',
                                       ]]

In [59]:
tdf.shape

(50, 12)

In [57]:
def fsq(lat,lon):
    url = "https://api.foursquare.com/v3/places/search"
    #base = "46.205,6.143" #Geneva Center

    headers = {
        "Accept": "application/json",
        "Authorization": "fsq3V3VJ3Eo+Pxwu5vUFPK9qfRQiPyRblc7ovL8NXNqZIvg="
    }
    
    par = {
        'll':f'{lat},{lon}',
        'radius':100,
        'category':13065,
        'limit':50,
        'fields':"name,fsq_id,categories,price,rating,stats,location,geocodes"
    }

    response = requests.get(url, headers=headers, params=par).json()
    return pd.json_normalize(response['results'])[[\
                                                   'fsq_id',
                                                   'categories',
                                                   'name',
                                                   'price',
                                                   'rating',
                                                   'stats.total_ratings',
                                                   'geocodes.main.latitude',
                                                   'geocodes.main.longitude',
                                                   'location.formatted_address',
                                                   'location.locality',
                                                   #'location.neighborhood',
                                                   'location.postcode',
                                                   'location.region',
                                                   ]]

In [41]:
# let's use our great df, we created in Grid Generator notebook
df_grid_points_final = pd.read_excel('../raw_data/raw_data_geneva_grid_points.xlsx', usecols='B:D') 
#df_grid_points_final

# convert grid points to numpy array for an efficient iteration which is far more efficient than iterrows()
grid_points_array = df_grid_points_final[['lat','long']].to_numpy()
print(grid_points_array.shape)
grid_points_array

(3245, 2)


array([[46.20151452,  6.11101004],
       [46.20213627,  6.11101004],
       [46.20337973,  6.11101004],
       ...,
       [46.19840571,  6.17658706],
       [46.19902748,  6.17658706],
       [46.19902748,  6.17748538]])

In [62]:
%%time

df_restaurants = tdf

for row in grid_points_array:
    lat, lon = row[0], row[1]
    try:
        df_restaurants = pd.concat([df_restaurants,fsq(lat,lon)])
        print(df_restaurants.shape)
    except:
        print('error')
        continue

(68, 12)
(87, 12)
(100, 12)
(107, 12)
(157, 12)
(185, 12)
(214, 12)
(242, 12)
(271, 12)
(287, 12)
(311, 12)
(332, 12)
(360, 12)
(390, 12)
(418, 12)
(463, 12)
(493, 12)
(525, 12)
(549, 12)
(589, 12)
(592, 12)
(597, 12)
(602, 12)
(621, 12)
(656, 12)
(673, 12)
(691, 12)
(710, 12)
(714, 12)
(718, 12)
(723, 12)
(772, 12)
(822, 12)
(845, 12)
(867, 12)
(881, 12)
(903, 12)
(923, 12)
(929, 12)
(933, 12)
(959, 12)
(1009, 12)
(1059, 12)
(1109, 12)
(1125, 12)
(1163, 12)
(1204, 12)
(1239, 12)
(1254, 12)
(1290, 12)
(1340, 12)
(1390, 12)
(1440, 12)
(1490, 12)
(1540, 12)
(1589, 12)
(1639, 12)
(1689, 12)
(1738, 12)
(1788, 12)
(1810, 12)
(1855, 12)
(1905, 12)
(1955, 12)
(2005, 12)
(2055, 12)
(2105, 12)
(2150, 12)
(2186, 12)
(2231, 12)
(2254, 12)
(2304, 12)
(2354, 12)
(2389, 12)
(2416, 12)
(2450, 12)
(2493, 12)
(2527, 12)
(2551, 12)
(2593, 12)
(2640, 12)
(2690, 12)
(2740, 12)
(2790, 12)
(2829, 12)
(2866, 12)
(2909, 12)
(2959, 12)
(3002, 12)
(3037, 12)
(3067, 12)
(3102, 12)
(3127, 12)
(3177, 12)
(3227, 12

(29775, 12)
(29825, 12)
(29875, 12)
(29925, 12)
(29975, 12)
(30025, 12)
(30075, 12)
(30125, 12)
(30175, 12)
(30225, 12)
(30275, 12)
(30325, 12)
(30375, 12)
(30425, 12)
(30475, 12)
(30525, 12)
(30575, 12)
(30625, 12)
(30675, 12)
(30725, 12)
(30775, 12)
(30821, 12)
(30862, 12)
(30867, 12)
(30873, 12)
(30875, 12)
(30879, 12)
(30882, 12)
(30931, 12)
(30981, 12)
(31031, 12)
(31081, 12)
(31131, 12)
(31181, 12)
(31227, 12)
(31273, 12)
(31323, 12)
(31373, 12)
(31423, 12)
(31473, 12)
(31523, 12)
(31573, 12)
(31623, 12)
(31673, 12)
(31723, 12)
(31773, 12)
(31823, 12)
(31873, 12)
(31923, 12)
(31973, 12)
(32023, 12)
(32073, 12)
(32123, 12)
(32173, 12)
(32223, 12)
(32273, 12)
(32323, 12)
(32373, 12)
(32423, 12)
(32473, 12)
(32523, 12)
(32531, 12)
(32544, 12)
(32556, 12)
(32588, 12)
(32638, 12)
(32644, 12)
(32650, 12)
(32657, 12)
(32683, 12)
(32717, 12)
(32763, 12)
(32813, 12)
(32858, 12)
(32908, 12)
(32958, 12)
(33008, 12)
(33058, 12)
(33108, 12)
(33158, 12)
(33208, 12)
(33258, 12)
(33308, 12)
(333

In [69]:
df_restaurants['categories']

0     [{'id': 11045, 'name': 'Bank', 'icon': {'prefi...
1     [{'id': 17051, 'name': 'Watch Store', 'icon': ...
2     [{'id': 11045, 'name': 'Bank', 'icon': {'prefi...
3                                                    []
4     [{'id': 19028, 'name': 'Tourist Information an...
                            ...                        
22                                                   []
23    [{'id': 11000, 'name': 'Business and Professio...
25    [{'id': 11148, 'name': 'Property Management Of...
27    [{'id': 11148, 'name': 'Property Management Of...
28    [{'id': 11149, 'name': 'Real Estate Agency', '...
Name: categories, Length: 19246, dtype: object

In [ ]:
# Sample coordinate
lat, lng = grid_points_array[1][0], grid_points_array[1][1]
print(lat, lng)

# Sample response / Google nearby search
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

params = {
    'keyword':'restaurant',
    'location':f'{lat},{lng}',
    'radius':100,
    'region':'ch',
    'key':google_api_access_key
}

response_sample = requests.get(base_url, params=params).json()
df_sample = pd.json_normalize(response_sample['results'])\
        [['name',
          'place_id',
          'types',
          'price_level',
          'rating',
          'user_ratings_total',
          'vicinity',
          'plus_code.global_code',
          'geometry.location.lat',
          'geometry.location.lng']]
df_sample

In [65]:
df_restaurants.drop_duplicates()

(49758, 12)

In [66]:
df_restaurants.drop_duplicates(subset='fsq_id',inplace=True)

In [67]:
df_restaurants.shape

(19246, 12)

In [68]:
df_restaurants.to_csv('../raw_data/FourSquare.csv', encoding='utf-8-sig')